In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playbyplay
jordan_id = 893
jordan_team_id = 1610612741

In [2]:
# Query for the last regular season game where the Pacers were playing
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType

gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = jordan_team_id,
                            season_nullable=Season.default,
                            season_type_nullable=SeasonType.regular)  

games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game = games[0]
game_id = game['GAME_ID']
game_matchup = game['MATCHUP']

In [3]:
from nba_api.stats.endpoints import playbyplay
df = playbyplay.PlayByPlay(game_id).get_data_frames()[0]
df.head(10)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
0,0022201223,2,12,0,1,1:13 PM,12:00,None,Start of 1st Period (1:13 PM EST),None,None,None
1,0022201223,4,10,0,1,1:13 PM,12:00,Jump Ball Vucevic vs. Wiseman: Tip to LaVine,None,None,None,None
2,0022201223,7,1,47,1,1:13 PM,11:40,DeRozan 9' Turnaround Jump Shot (2 PTS),None,None,0 - 2,2
3,0022201223,9,1,6,1,1:14 PM,11:18,None,None,Hayes 2' Driving Layup (2 PTS),2 - 2,TIE
4,0022201223,10,2,1,1,1:14 PM,11:02,MISS LaVine 21' Jump Shot,None,None,None,None
5,0022201223,11,4,0,1,1:14 PM,10:56,None,None,Hayes REBOUND (Off:0 Def:1),None,None
6,0022201223,12,2,1,1,1:14 PM,10:53,None,None,MISS Ivey 27' 3PT Jump Shot,None,None
7,0022201223,13,4,0,1,1:14 PM,10:51,DeRozan REBOUND (Off:0 Def:1),None,None,None,None
8,0022201223,14,1,52,1,1:14 PM,10:48,LaVine Alley Oop Dunk (2 PTS) (DeRozan 1 AST),None,None,2 - 4,2
9,0022201223,16,6,1,1,1:14 PM,10:32,Caruso P.FOUL (P1.T1) (M.Smith),None,None,None,None


In [4]:
import pandas
pandas.set_option('display.max_colwidth',500)
pandas.set_option('display.max_rows',500)

In [5]:
from enum import Enum

class EventMsgType(Enum):
    FIELD_GOAL_MADE = 1
    FIELD_GOAL_MISSED = 2
    FREE_THROWfree_throw_attempt = 3
    REBOUND = 4
    TURNOVER = 5
    FOUL = 6
    VIOLATION = 7
    SUBSTITUTION = 8
    TIMEOUT = 9
    JUMP_BALL = 10
    EJECTION = 11
    PERIOD_BEGIN = 12
    PERIOD_END = 13

In [6]:
df.loc[df['EVENTMSGTYPE'] == 4].head(5)

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN
5,0022201223,11,4,0,1,1:14 PM,10:56,None,None,Hayes REBOUND (Off:0 Def:1),None,None
7,0022201223,13,4,0,1,1:14 PM,10:51,DeRozan REBOUND (Off:0 Def:1),None,None,None,None
11,0022201223,20,4,0,1,1:15 PM,10:18,Caruso REBOUND (Off:0 Def:1),None,None,None,None
14,0022201223,24,4,0,1,1:15 PM,10:00,None,None,Wiseman REBOUND (Off:1 Def:0),None,None
20,0022201223,34,4,0,1,1:17 PM,8:53,None,None,Wiseman REBOUND (Off:2 Def:0),None,None


In [7]:
import re
import operator

#the following expression is specific to EventMsgType 1
p = re.compile('(\s{2}|\' )([\w+ ]+)')

#get the PlayByPlay data from the Pacers game_id
plays = playbyplay.PlayByPlay(game_id).get_normalized_dict()['PlayByPlay']

#declare a few variables
description = ''
event_msg_action_types = {}

#loop over the play by play data
for play in plays:
    if play['EVENTMSGTYPE'] == 1:
        if play['HOMEDESCRIPTION']:
            description = play['HOMEDESCRIPTION']
        else:
            description = play['VISITORDESCRIPTION']
        if p.search(description):
            #do a bit of searching(regex) and a little character magic: underscores and upper case
            event_msg_action = re.sub(' ', '_', p.search(description).groups()[1].rstrip()).upper()
            #Add it to our dictionary
            event_msg_action_types[event_msg_action] = play['EVENTMSGACTIONTYPE']
            
#sort it all
event_msg_action_types = sorted(event_msg_action_types.items(), key=operator.itemgetter(0))

#output a class that we could plug into our code base
for action in event_msg_action_types:
    print(f'\t{action[0]} = {action[1]}')

	3PT_JUMP_SHOT = 1
	3PT_PULLUP_JUMP_SHOT = 79
	3PT_STEP_BACK_JUMP_SHOT = 80
	CUTTING_LAYUP_SHOT = 98
	DRIVING_FINGER_ROLL_LAYUP = 75
	DRIVING_FLOATING_BANK_JUMP_SHOT = 102
	DRIVING_FLOATING_JUMP_SHOT = 101
	DRIVING_HOOK_SHOT = 57
	DRIVING_LAYUP = 6
	DUNK = 7
	FLOATING_JUMP_SHOT = 78
	JUMP_SHOT = 1
	LAYUP = 5
	PULLUP_JUMP_SHOT = 79
	RUNNING_LAYUP = 41
	STEP_BACK_JUMP_SHOT = 80
	TURNAROUND_HOOK_SHOT = 58
	TURNAROUND_JUMP_SHOT = 47


In [8]:
#get the PlayByPlay data from the Pacers game_id
plays = playbyplay.PlayByPlay(game_id).get_normalized_dict()['PlayByPlay']

#declare a few variables
description = ''
event_msg_action_types = {}

#loop over the play by play data
#do a bit of findall(regex) and a little character magic: underscores and upper case
#we're using a findall here as we have to deal with the extra word MISS at the beginning of the text.
#that extra text means we'll have multiple matches for our regex.
for play in plays:
    if play['EVENTMSGTYPE'] == 2:
        match = list()
        if play['HOMEDESCRIPTION'] is not None: 
            match = p.findall(play['HOMEDESCRIPTION'])

            #looking for blocks
            if len(match) & (play['VISITORDESCRIPTION'] is not None):
                print(play['VISITORDESCRIPTION'])

        else:
            match = p.findall(play['VISITORDESCRIPTION'])
            
            #looking for blocks
            if len(match) & (play['HOMEDESCRIPTION'] is not None):
                print(play['HOMEDESCRIPTION'])
        if match:
            event_msg_action = re.sub(' ', '_', match[0][1]).upper()
            event_msg_action_types[event_msg_action] = play['EVENTMSGACTIONTYPE']
            
event_msg_action_types = sorted(event_msg_action_types.items(), key=operator.itemgetter(0))

print("")
for action in event_msg_action_types:
    print(f'{action[0]} = {action[1]}')

Wiseman BLOCK (1 BLK)
Duren BLOCK (1 BLK)
Hayes BLOCK (1 BLK)

3PT_JUMP_SHOT = 1
3PT_PULLUP_JUMP_SHOT = 79
3PT_STEP_BACK_JUMP_SHOT = 80
ALLEY_OOP_LAYUP = 43
DRIVING_DUNK = 9
DRIVING_FLOATING_BANK_JUMP_SHOT = 102
DRIVING_FLOATING_JUMP_SHOT = 101
DRIVING_HOOK_SHOT = 57
DRIVING_LAYUP = 6
DRIVING_REVERSE_LAYUP = 73
FADEAWAY_JUMPER = 63
FLOATING_JUMP_SHOT = 78
JUMP_SHOT = 1
LAYUP = 5
PULLUP_JUMP_SHOT = 79
PUTBACK_LAYUP = 72
RUNNING_LAYUP = 41
TIP_LAYUP_SHOT = 97
TURNAROUND_JUMP_SHOT = 47


In [10]:
play = pd.read_csv("play_by_play.csv")
play.columns

Index(['game_id', 'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'period',
       'wctimestring', 'pctimestring', 'homedescription', 'neutraldescription',
       'visitordescription', 'score', 'scoremargin', 'person1type',
       'player1_id', 'player1_name', 'player1_team_id', 'player1_team_city',
       'player1_team_nickname', 'player1_team_abbreviation', 'person2type',
       'player2_id', 'player2_name', 'player2_team_id', 'player2_team_city',
       'player2_team_nickname', 'player2_team_abbreviation', 'person3type',
       'player3_id', 'player3_name', 'player3_team_id', 'player3_team_city',
       'player3_team_nickname', 'player3_team_abbreviation',
       'video_available_flag'],
      dtype='object')

In [11]:
play["player1_name"]

0                          NaN
1              Hakeem Olajuwon
2           Corliss Williamson
3              Hakeem Olajuwon
4                Clyde Drexler
                   ...        
13227006             Ish Smith
13227007       Cameron Johnson
13227008          Nikola Jokic
13227009                   NaN
13227010                   NaN
Name: player1_name, Length: 13227011, dtype: object

In [12]:
play = play.drop(['player1_name', 'player1_team_abbreviation', 'player2_team_city',
       'player2_team_nickname', 'player2_team_abbreviation', 'player3_name', 'player3_team_id', 'player3_team_city',
       'player3_team_nickname', 'player3_team_abbreviation', 'video_available_flag', 'person1type', 'person2type', 'person3type'], axis = 1)

In [14]:
play.head(20)

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,score,scoremargin,player1_id,player1_team_id,player1_team_city,player1_team_nickname,player2_id,player2_name,player2_team_id,player3_id
0,29600009,2,12,0,1,11:44 PM,12:00,NaN,Start of 1st Period (11:44 PM EST),NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0
1,29600009,3,10,0,1,11:45 PM,12:00,Jump Ball Olajuwon vs. Polynice: Tip to Williamson,NaN,NaN,NaN,NaN,165,1.610613e+09,Houston,Rockets,178,Olden Polynice,1.610613e+09,722
2,29600009,4,5,4,1,11:46 PM,11:49,NaN,NaN,Williamson Traveling Turnover (P1.T1),NaN,NaN,722,1.610613e+09,Sacramento,Kings,0,NaN,NaN,0
3,29600009,5,2,1,1,11:46 PM,11:27,MISS Olajuwon 13' Jump Shot,NaN,NaN,NaN,NaN,165,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
4,29600009,7,4,0,1,11:46 PM,11:23,Drexler REBOUND (Off:1 Def:0),NaN,NaN,NaN,NaN,17,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
5,29600009,6,2,5,1,11:46 PM,11:23,MISS Drexler Layup,NaN,NaN,NaN,NaN,17,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
6,29600009,8,4,0,1,11:46 PM,11:22,Olajuwon REBOUND (Off:1 Def:0),NaN,NaN,NaN,NaN,165,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
7,29600009,9,1,2,1,11:46 PM,11:18,Elie 10' Running Jump Shot (2 PTS) (Olajuwon 1 AST),NaN,NaN,0 - 2,2,53,1.610613e+09,Houston,Rockets,165,Hakeem Olajuwon,1.610613e+09,0
8,29600009,10,1,1,1,11:47 PM,11:00,NaN,NaN,Abdul-Rauf 9' Jump Shot (2 PTS),2 - 2,TIE,51,1.610613e+09,Sacramento,Kings,0,NaN,NaN,0
9,29600009,11,1,6,1,11:47 PM,10:44,Maloney Driving Layup (2 PTS) (Drexler 1 AST),NaN,NaN,2 - 4,2,1074,1.610613e+09,Houston,Rockets,17,Clyde Drexler,1.610613e+09,0


In [16]:
play

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,score,scoremargin,player1_id,player1_team_id,player1_team_city,player1_team_nickname,player2_id,player2_name,player2_team_id,player3_id
0,29600009,2,12,0,1,11:44 PM,12:00,NaN,Start of 1st Period (11:44 PM EST),NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0
1,29600009,3,10,0,1,11:45 PM,12:00,Jump Ball Olajuwon vs. Polynice: Tip to Williamson,NaN,NaN,NaN,NaN,165,1.610613e+09,Houston,Rockets,178,Olden Polynice,1.610613e+09,722
2,29600009,4,5,4,1,11:46 PM,11:49,NaN,NaN,Williamson Traveling Turnover (P1.T1),NaN,NaN,722,1.610613e+09,Sacramento,Kings,0,NaN,NaN,0
3,29600009,5,2,1,1,11:46 PM,11:27,MISS Olajuwon 13' Jump Shot,NaN,NaN,NaN,NaN,165,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
4,29600009,7,4,0,1,11:46 PM,11:23,Drexler REBOUND (Off:1 Def:0),NaN,NaN,NaN,NaN,17,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13227006,22201014,662,8,0,4,6:01 PM,0:01,SUB: Porter Jr. FOR Smith,NaN,NaN,NaN,NaN,202397,1.610613e+09,Denver,Nuggets,1629008,Michael Porter Jr.,1.610613e+09,0
13227007,22201014,664,8,0,4,6:01 PM,0:01,NaN,NaN,SUB: Claxton FOR Johnson,NaN,NaN,1629661,1.610613e+09,Brooklyn,Nets,1629651,Nic Claxton,1.610613e+09,0
13227008,22201014,666,2,47,4,6:02 PM,0:00,MISS Jokic 25' 3PT Turnaround Jump Shot,NaN,NaN,NaN,NaN,203999,1.610613e+09,Denver,Nuggets,0,NaN,NaN,0
13227009,22201014,667,4,0,4,6:02 PM,0:00,NUGGETS Rebound,NaN,NaN,NaN,NaN,1610612743,NaN,NaN,NaN,0,NaN,NaN,0


In [17]:
play[play["eventmsgtype"] == 11]

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,score,scoremargin,player1_id,player1_team_id,player1_team_city,player1_team_nickname,player2_id,player2_name,player2_team_id,player3_id
6911,29600022,362,11,3,4,12:52 PM,9:59,Horry Ejection:First Flagrant Type 2,NaN,NaN,NaN,NaN,109,1.610613e+09,Phoenix,Suns,0,NaN,NaN,0
20632,29600052,396,11,1,4,15:59 PM,8:09,Robinson Ejection:Second Technical,NaN,NaN,NaN,NaN,361,1.610613e+09,Portland,Trail Blazers,0,NaN,NaN,0
69794,29600166,428,11,1,4,11:10 PM,0:13,NaN,NaN,Rodman Ejection:Second Technical,NaN,NaN,23,1.610613e+09,Chicago,Bulls,0,NaN,NaN,0
70984,29600161,276,11,1,3,13:07 PM,2:34,NaN,Ejection:Second Technical,NaN,NaN,NaN,1242,NaN,NaN,NaN,0,NaN,NaN,0
92187,29600213,164,11,1,2,12:08 PM,11:11,NaN,Ejection:Second Technical,NaN,NaN,NaN,1879048195,NaN,NaN,NaN,0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13184571,22200921,561,11,4,4,9:53 PM,3:46,NaN,NaN,Tatum Ejection:Other,NaN,NaN,1628369,1.610613e+09,Boston,Celtics,0,NaN,NaN,0
13206657,22200975,634,11,4,4,11:20 PM,0:28,NaN,NaN,Barnes Ejection:Other,NaN,NaN,1630567,1.610613e+09,Toronto,Raptors,0,NaN,NaN,0
13218008,22201002,421,11,4,3,9:38 PM,6:57,NaN,NaN,Porter Jr. Ejection:Other,NaN,NaN,1629008,1.610613e+09,Denver,Nuggets,0,NaN,NaN,0
13218009,22201002,422,11,4,3,9:38 PM,6:57,Collins Ejection:Other,NaN,NaN,NaN,NaN,1628380,1.610613e+09,San Antonio,Spurs,0,NaN,NaN,0


In [18]:
temp = play.loc[(play["eventmsgtype"] == 8)]

In [19]:
play.head(50)

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,wctimestring,pctimestring,homedescription,neutraldescription,visitordescription,score,scoremargin,player1_id,player1_team_id,player1_team_city,player1_team_nickname,player2_id,player2_name,player2_team_id,player3_id
0,29600009,2,12,0,1,11:44 PM,12:00,NaN,Start of 1st Period (11:44 PM EST),NaN,NaN,NaN,0,NaN,NaN,NaN,0,NaN,NaN,0
1,29600009,3,10,0,1,11:45 PM,12:00,Jump Ball Olajuwon vs. Polynice: Tip to Williamson,NaN,NaN,NaN,NaN,165,1.610613e+09,Houston,Rockets,178,Olden Polynice,1.610613e+09,722
2,29600009,4,5,4,1,11:46 PM,11:49,NaN,NaN,Williamson Traveling Turnover (P1.T1),NaN,NaN,722,1.610613e+09,Sacramento,Kings,0,NaN,NaN,0
3,29600009,5,2,1,1,11:46 PM,11:27,MISS Olajuwon 13' Jump Shot,NaN,NaN,NaN,NaN,165,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
4,29600009,7,4,0,1,11:46 PM,11:23,Drexler REBOUND (Off:1 Def:0),NaN,NaN,NaN,NaN,17,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
5,29600009,6,2,5,1,11:46 PM,11:23,MISS Drexler Layup,NaN,NaN,NaN,NaN,17,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
6,29600009,8,4,0,1,11:46 PM,11:22,Olajuwon REBOUND (Off:1 Def:0),NaN,NaN,NaN,NaN,165,1.610613e+09,Houston,Rockets,0,NaN,NaN,0
7,29600009,9,1,2,1,11:46 PM,11:18,Elie 10' Running Jump Shot (2 PTS) (Olajuwon 1 AST),NaN,NaN,0 - 2,2,53,1.610613e+09,Houston,Rockets,165,Hakeem Olajuwon,1.610613e+09,0
8,29600009,10,1,1,1,11:47 PM,11:00,NaN,NaN,Abdul-Rauf 9' Jump Shot (2 PTS),2 - 2,TIE,51,1.610613e+09,Sacramento,Kings,0,NaN,NaN,0
9,29600009,11,1,6,1,11:47 PM,10:44,Maloney Driving Layup (2 PTS) (Drexler 1 AST),NaN,NaN,2 - 4,2,1074,1.610613e+09,Houston,Rockets,17,Clyde Drexler,1.610613e+09,0
